# Phase 2.1: Optimized Kernels Benchmark

This notebook tests Phase 2.1 optimizations that fix the Phase 2 regression:

**Phase 2 Issues Identified:**
- "Fused" kernel wasn't truly fused (stored intermediate in shared memory)
- atomicAdd in T-MAC LUT causing contention
- Overhead from stream sync and pinned memory for small tensors

**Phase 2.1 Solutions:**
1. **Warp-private accumulation** - No atomicAdd (each thread accumulates privately)
2. **True streaming fusion** - Normalize on-the-fly without intermediate storage
3. **Adaptive dispatch** - Choose best kernel based on tensor size
4. **Warp-level shuffle reduction** - Fast reduction using `__shfl_down_sync`

**Target**: Fix Phase 2 regression, achieve 150+ tok/s (baseline before: 80 tok/s)

**Requirements:**
- NVIDIA GPU (T4, RTX, Jetson)
- CUDA Toolkit 11.0+

## 1. Environment Setup

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

In [ ]:
# Clone repository
import os
if not os.path.exists('ollama-api-gateway'):
    !git clone https://github.com/umerkhan95/ollama-api-gateway.git
else:
    print('Repository exists, pulling latest...')
    !cd ollama-api-gateway && git pull

In [ ]:
# Build CUDA kernels with Phase 2.1 optimizations
%cd ollama-api-gateway/mojo-gateway/src/kernels
!make cuda
!ls -la ../../lib/

## 2. Load CUDA Library with Phase 2.1 APIs

In [ ]:
import ctypes
import numpy as np
import time

# Load library
lib_path = '../../lib/libtmac_kernel_cuda.so'
cuda_lib = ctypes.CDLL(lib_path)
print(f'Loaded: {lib_path}')

# Basic functions
cuda_lib.cuda_available.restype = ctypes.c_int
cuda_lib.cuda_device_name.restype = ctypes.c_char_p
cuda_lib.cuda_init.argtypes = [ctypes.c_int, ctypes.c_int, ctypes.c_int]
cuda_lib.cuda_init.restype = ctypes.c_int
cuda_lib.cuda_cleanup.restype = None
cuda_lib.cuda_sync.restype = None

# Phase 1: Persistent memory API
cuda_lib.cuda_load_weights.argtypes = [
    ctypes.POINTER(ctypes.c_int8), ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int
]
cuda_lib.cuda_load_weights.restype = ctypes.c_int
cuda_lib.cuda_unload_weights.restype = None
cuda_lib.cuda_weights_loaded.restype = ctypes.c_int

cuda_lib.tmac_matmul_cuda_persistent.argtypes = [
    ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int, ctypes.c_int
]
cuda_lib.tmac_matmul_cuda_persistent.restype = ctypes.c_int

cuda_lib.cuda_load_norm_weights.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.c_int]
cuda_lib.cuda_load_norm_weights.restype = ctypes.c_int

cuda_lib.rmsnorm_cuda_persistent.argtypes = [
    ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int, ctypes.c_float
]
cuda_lib.rmsnorm_cuda_persistent.restype = ctypes.c_int

# Phase 2: Original fused kernels (for comparison)
cuda_lib.fused_rmsnorm_matmul_cuda.argtypes = [
    ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_float
]
cuda_lib.fused_rmsnorm_matmul_cuda.restype = ctypes.c_int

# Phase 2.1: NEW Optimized kernels
# V3 kernel (warp-private accumulation, no atomicAdd)
cuda_lib.tmac_matmul_cuda_v3.argtypes = [
    ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int, ctypes.c_int
]
cuda_lib.tmac_matmul_cuda_v3.restype = ctypes.c_int

# Streaming fused (true fusion, normalize on-the-fly)
cuda_lib.streaming_fused_rmsnorm_matmul_cuda.argtypes = [
    ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int, ctypes.c_float
]
cuda_lib.streaming_fused_rmsnorm_matmul_cuda.restype = ctypes.c_int

# Adaptive dispatch (auto-selects best kernel)
cuda_lib.tmac_matmul_cuda_adaptive.argtypes = [
    ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int, ctypes.c_int
]
cuda_lib.tmac_matmul_cuda_adaptive.restype = ctypes.c_int

cuda_lib.fused_rmsnorm_matmul_cuda_adaptive.argtypes = [
    ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float),
    ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_float
]
cuda_lib.fused_rmsnorm_matmul_cuda_adaptive.restype = ctypes.c_int

print('Function signatures defined')

In [ ]:
# Check CUDA and initialize
if cuda_lib.cuda_available():
    device_name = cuda_lib.cuda_device_name().decode('utf-8')
    print(f'CUDA Device: {device_name}')
else:
    raise RuntimeError('No CUDA device found!')

# Initialize with generous buffer sizes
max_weights = 100_000_000  # 100MB for weights
max_activations = 10_000_000
max_output = 10_000_000
ret = cuda_lib.cuda_init(max_weights, max_activations, max_output)
if ret == 0:
    print('CUDA initialized successfully')
else:
    raise RuntimeError('CUDA initialization failed')

## 3. Model Parameters (SmolLM-135M)

In [ ]:
# SmolLM-135M architecture
hidden_size = 576
intermediate_size = 1536
num_heads = 9
head_dim = hidden_size // num_heads  # 64
vocab_size = 49152
num_layers = 9

print(f'Hidden size: {hidden_size}')
print(f'Intermediate size: {intermediate_size}')
print(f'Num layers: {num_layers}')
print(f'\nTest tensor sizes (M*K):')
print(f'  QKV Projection: {3*hidden_size} x {hidden_size} = {3*hidden_size*hidden_size:,}')
print(f'  FFN Up:         {intermediate_size} x {hidden_size} = {intermediate_size*hidden_size:,}')
print(f'  FFN Down:       {hidden_size} x {intermediate_size} = {hidden_size*intermediate_size:,}')

## 4. Benchmark Functions

In [ ]:
def benchmark_phase1_separate(weights, activations, output, scales, norm_weights, M, N, K, iterations=100):
    """Phase 1: Separate RMSNorm + MatMul calls."""
    weights_ptr = weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    act_ptr = activations.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    scales_ptr = scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    weight_bytes = M * ((K + 3) // 4)
    norm_out = np.zeros(K, dtype=np.float32)
    norm_out_ptr = norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    cuda_lib.cuda_load_weights(weights_ptr, scales_ptr, weight_bytes, M)
    cuda_lib.cuda_load_norm_weights(norm_ptr, K)
    
    # Warmup
    for _ in range(10):
        cuda_lib.rmsnorm_cuda_persistent(norm_out_ptr, act_ptr, 1, K, ctypes.c_float(1e-6))
        cuda_lib.tmac_matmul_cuda_persistent(norm_out_ptr, out_ptr, M, N, K)
    cuda_lib.cuda_sync()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.rmsnorm_cuda_persistent(norm_out_ptr, act_ptr, 1, K, ctypes.c_float(1e-6))
        cuda_lib.tmac_matmul_cuda_persistent(norm_out_ptr, out_ptr, M, N, K)
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000

def benchmark_phase2_fused(weights, activations, output, scales, norm_weights, M, N, K, iterations=100):
    """Phase 2: Original fused kernel (has atomicAdd issues)."""
    weights_ptr = weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    act_ptr = activations.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    scales_ptr = scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    weight_bytes = M * ((K + 3) // 4)
    cuda_lib.cuda_load_weights(weights_ptr, scales_ptr, weight_bytes, M)
    cuda_lib.cuda_load_norm_weights(norm_ptr, K)
    
    for _ in range(10):
        cuda_lib.fused_rmsnorm_matmul_cuda(act_ptr, out_ptr, M, N, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync()
    
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.fused_rmsnorm_matmul_cuda(act_ptr, out_ptr, M, N, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000

def benchmark_v3_kernel(weights, activations, output, scales, norm_weights, M, N, K, iterations=100):
    """Phase 2.1: V3 kernel (warp-private accumulation, no atomicAdd)."""
    weights_ptr = weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    act_ptr = activations.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    scales_ptr = scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    weight_bytes = M * ((K + 3) // 4)
    norm_out = np.zeros(K, dtype=np.float32)
    norm_out_ptr = norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    cuda_lib.cuda_load_weights(weights_ptr, scales_ptr, weight_bytes, M)
    cuda_lib.cuda_load_norm_weights(norm_ptr, K)
    
    for _ in range(10):
        cuda_lib.rmsnorm_cuda_persistent(norm_out_ptr, act_ptr, 1, K, ctypes.c_float(1e-6))
        cuda_lib.tmac_matmul_cuda_v3(norm_out_ptr, out_ptr, M, N, K)
    cuda_lib.cuda_sync()
    
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.rmsnorm_cuda_persistent(norm_out_ptr, act_ptr, 1, K, ctypes.c_float(1e-6))
        cuda_lib.tmac_matmul_cuda_v3(norm_out_ptr, out_ptr, M, N, K)
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000

def benchmark_streaming_fused(weights, activations, output, scales, norm_weights, M, N, K, iterations=100):
    """Phase 2.1: Streaming fused kernel (true fusion, normalize on-the-fly)."""
    weights_ptr = weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    act_ptr = activations.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    scales_ptr = scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    weight_bytes = M * ((K + 3) // 4)
    cuda_lib.cuda_load_weights(weights_ptr, scales_ptr, weight_bytes, M)
    cuda_lib.cuda_load_norm_weights(norm_ptr, K)
    
    for _ in range(10):
        cuda_lib.streaming_fused_rmsnorm_matmul_cuda(act_ptr, out_ptr, M, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync()
    
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.streaming_fused_rmsnorm_matmul_cuda(act_ptr, out_ptr, M, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000

def benchmark_adaptive(weights, activations, output, scales, norm_weights, M, N, K, iterations=100):
    """Phase 2.1: Adaptive dispatch (auto-selects best kernel)."""
    weights_ptr = weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    act_ptr = activations.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    scales_ptr = scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    weight_bytes = M * ((K + 3) // 4)
    cuda_lib.cuda_load_weights(weights_ptr, scales_ptr, weight_bytes, M)
    cuda_lib.cuda_load_norm_weights(norm_ptr, K)
    
    for _ in range(10):
        cuda_lib.fused_rmsnorm_matmul_cuda_adaptive(act_ptr, out_ptr, M, N, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync()
    
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.fused_rmsnorm_matmul_cuda_adaptive(act_ptr, out_ptr, M, N, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000

## 5. Run Benchmarks: Phase 1 vs Phase 2 vs Phase 2.1

In [ ]:
# Test configurations simulating transformer layers
test_configs = [
    ('QKV Projection', 3 * hidden_size, 1, hidden_size),
    ('Output Projection', hidden_size, 1, hidden_size),
    ('FFN Up', intermediate_size, 1, hidden_size),
    ('FFN Down', hidden_size, 1, intermediate_size),
]

print('=' * 130)
print('PHASE 2.1 BENCHMARK: Optimized Kernels (No Atomics, True Fusion)')
print('=' * 130)
print(f'{"Layer":<18} {"M*K":>10} {"Phase1":>10} {"Phase2":>10} {"V3":>10} {"Stream":>10} {"Adaptive":>10} {"Best":>10} {"vs Phase1":>10}')
print(f'{"":<18} {"":>10} {"(ms)":>10} {"(ms)":>10} {"(ms)":>10} {"(ms)":>10} {"(ms)":>10} {"":>10} {"":>10}')
print('-' * 130)

results = []
iterations = 100

for name, M, N, K in test_configs:
    weight_bytes = M * ((K + 3) // 4)
    weights = np.random.randint(-1, 2, size=weight_bytes, dtype=np.int8)
    activations = np.random.randn(K * N).astype(np.float32)
    output = np.zeros(M * N, dtype=np.float32)
    scales = np.ones(M, dtype=np.float32)
    norm_weights = np.ones(K, dtype=np.float32)
    
    try:
        phase1_ms = benchmark_phase1_separate(weights, activations, output, scales, norm_weights, M, N, K, iterations)
        phase2_ms = benchmark_phase2_fused(weights, activations, output, scales, norm_weights, M, N, K, iterations)
        v3_ms = benchmark_v3_kernel(weights, activations, output, scales, norm_weights, M, N, K, iterations)
        stream_ms = benchmark_streaming_fused(weights, activations, output, scales, norm_weights, M, N, K, iterations)
        adaptive_ms = benchmark_adaptive(weights, activations, output, scales, norm_weights, M, N, K, iterations)
        
        best_ms = min(phase1_ms, phase2_ms, v3_ms, stream_ms, adaptive_ms)
        best_name = ['Phase1', 'Phase2', 'V3', 'Stream', 'Adaptive'][[phase1_ms, phase2_ms, v3_ms, stream_ms, adaptive_ms].index(best_ms)]
        speedup = phase1_ms / best_ms
        
        print(f'{name:<18} {M*K:>10,} {phase1_ms:>10.3f} {phase2_ms:>10.3f} {v3_ms:>10.3f} {stream_ms:>10.3f} {adaptive_ms:>10.3f} {best_name:>10} {speedup:>9.2f}x')
        results.append((name, M, K, phase1_ms, phase2_ms, v3_ms, stream_ms, adaptive_ms, best_ms, speedup))
    except Exception as e:
        print(f'{name:<18} ERROR: {e}')

print('-' * 130)

## 6. Analyze Results

In [ ]:
if results:
    print('\n' + '=' * 80)
    print('ANALYSIS: Which kernel is fastest for each layer?')
    print('=' * 80)
    
    for r in results:
        name, M, K, p1, p2, v3, stream, adaptive, best, speedup = r
        print(f'\n{name}:')
        print(f'  Phase 1 (separate):     {p1:.3f} ms')
        print(f'  Phase 2 (atomicAdd):    {p2:.3f} ms ({p1/p2:.2f}x vs Phase1)')
        print(f'  V3 (warp-private):      {v3:.3f} ms ({p1/v3:.2f}x vs Phase1)')
        print(f'  Streaming (true fuse):  {stream:.3f} ms ({p1/stream:.2f}x vs Phase1)')
        print(f'  Adaptive:               {adaptive:.3f} ms ({p1/adaptive:.2f}x vs Phase1)')
        print(f'  --> Best: {best:.3f} ms ({speedup:.2f}x speedup)')
    
    # Calculate totals
    total_phase1 = sum(r[3] for r in results)
    total_phase2 = sum(r[4] for r in results)
    total_v3 = sum(r[5] for r in results)
    total_stream = sum(r[6] for r in results)
    total_adaptive = sum(r[7] for r in results)
    
    print('\n' + '=' * 80)
    print('TOTAL TIME PER LAYER (all projections):')
    print('=' * 80)
    print(f'Phase 1:  {total_phase1:.3f} ms')
    print(f'Phase 2:  {total_phase2:.3f} ms ({total_phase1/total_phase2:.2f}x)')
    print(f'V3:       {total_v3:.3f} ms ({total_phase1/total_v3:.2f}x)')
    print(f'Streaming:{total_stream:.3f} ms ({total_phase1/total_stream:.2f}x)')
    print(f'Adaptive: {total_adaptive:.3f} ms ({total_phase1/total_adaptive:.2f}x)')

## 7. Estimated Throughput

In [ ]:
if results:
    # Use adaptive kernel for final estimate
    total_adaptive = sum(r[7] for r in results)
    total_phase1 = sum(r[3] for r in results)
    
    # Per-token time = layer_time * num_layers
    token_time_phase1 = total_phase1 * num_layers
    token_time_adaptive = total_adaptive * num_layers
    
    tok_s_phase1 = 1000 / token_time_phase1
    tok_s_adaptive = 1000 / token_time_adaptive
    
    print('\n' + '=' * 80)
    print('ESTIMATED THROUGHPUT (SmolLM-135M, 9 layers)')
    print('=' * 80)
    
    print(f'\nPhase 1 (Persistent Memory):')
    print(f'  Per token: {token_time_phase1:.2f} ms')
    print(f'  Throughput: {tok_s_phase1:.1f} tok/s')
    
    print(f'\nPhase 2.1 (Adaptive Dispatch):')
    print(f'  Per token: {token_time_adaptive:.2f} ms')
    print(f'  Throughput: {tok_s_adaptive:.1f} tok/s')
    
    print(f'\nImprovement: {tok_s_adaptive/tok_s_phase1:.2f}x faster than Phase 1')
    
    OLLAMA_TOK_S = 423
    print(f'\n' + '-' * 80)
    print(f'Ollama comparison:')
    print(f'  EdgeLLM Phase 2.1: {tok_s_adaptive:.1f} tok/s')
    print(f'  Ollama:            {OLLAMA_TOK_S} tok/s')
    print(f'  Gap:               {OLLAMA_TOK_S/tok_s_adaptive:.1f}x')
    print(f'  Progress:          {tok_s_adaptive/OLLAMA_TOK_S*100:.1f}% of Ollama speed')

## 8. Cleanup

In [ ]:
cuda_lib.cuda_cleanup()
print('CUDA resources cleaned up')

## 9. Summary

### Phase 2.1 Optimizations:

| Kernel | Key Feature | Best For |
|--------|-------------|----------|
| V3 | Warp-private accumulation, no atomicAdd | Medium tensors |
| Streaming | True fusion (normalize on-the-fly) | Large tensors, batch=1 |
| Adaptive | Auto-selects best kernel | All cases |

### Phase 2 vs Phase 2.1:

| Issue | Phase 2 Problem | Phase 2.1 Solution |
|-------|-----------------|--------------------|
| Fusion | Stored intermediate in shared mem | Normalize on-the-fly |
| Atomics | atomicAdd contention | Warp-private + shuffle reduction |
| Dispatch | One-size-fits-all | Adaptive based on tensor size |

### Next Steps:
- Profile with Nsight Compute to identify remaining bottlenecks
- Consider INT8 tensor core path for large matrices
- Implement CUDA Graphs for full forward pass